In [1]:
import pandas as pd
import faker
from faker import Faker
from faker.providers.phone_number import Provider
import mysql.connector as mysql
import os
import sys
import random
import numpy as np
import re

In [2]:
user = os.environ.get("USER")
password = os.environ.get("PASSWORD")
host = os.environ.get("HOST")
database = os.environ.get("DATABASE")

In [3]:
fake = Faker(["pl_PL"])


class PolishProfileProvider(Provider):
    """
    A Provider for polish profile.
    """

    def polish_phone_number(self):
        return f"+48 {self.msisdn()[:3]} {self.msisdn()[3:6]} {self.msisdn()[6:9]}"

    def polish_mail(self, first_name=None, last_name=None):

        if first_name is None:
            first_name = fake.name().split(" ")[0]
        if last_name is None:
            last_name = fake.name().split(" ")[1]

        polish_characters = {
            "ą": "a",
            "ć": "c",
            "ę": "e",
            "ł": "l",
            "ń": "n",
            "ó": "o",
            "ś": "s",
            "ź": "z",
            "ż": "z",
            "Ą": "A",
            "Ć": "C",
            "Ę": "E",
            "Ł": "L",
            "Ń": "N",
            "Ó": "O",
            "Ś": "S",
            "Ź": "Z",
            "Ż": "Z",
        }
        first_name = "".join(polish_characters.get(c, c) for c in first_name)
        last_name = "".join(polish_characters.get(c, c) for c in last_name)

        domain_name = np.random.choice(
            [
                "gmail.com",
                "onet.pl",
                "wp.pl",
                "interia.pl",
                "o2.pl",
                "yahoo.com",
                "outlook.com",
                "hotmail.com",
            ],
            p=[0.4, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05],
        )

        orders = [
            f"{first_name[:1].lower()}{last_name.lower()}@{domain_name}",
            f"{first_name.lower()}.{last_name.lower()}@{domain_name}",
            f"{first_name.lower()}{last_name.lower()}@{domain_name}",
            f"{first_name.lower()[:1]}.{last_name.lower()}@{domain_name}",
            f"{first_name.lower()}.{last_name.lower()[:1]}@{domain_name}",
            f"{first_name.lower()}{last_name.lower()[:1]}@{domain_name}",
        ]
        return np.random.choice(orders, p=[0.1, 0.3, 0.3, 0.1, 0.1, 0.1])

    def polish_profile(self):
        regex = re.compile(r".*pan.*")
        while True:
            profile = fake.profile()
            if not regex.match(profile["name"]):
                break
        first_name = profile["name"].split(" ")[0]
        last_name = profile["name"].split(" ")[1]
        address = profile["address"].replace("\n", " ")
        sex = "F" if first_name[-1] == "a" else "M"
        return {
            "first_name": first_name,
            "last_name": last_name,
            "phone_number": self.polish_phone_number(),
            "mail": self.polish_mail(first_name, last_name),
            "address": address,
            "sex": sex,
        }


fake.add_provider(PolishProfileProvider)

In [4]:
with mysql.connect(host=host, user=user, password=password, database=database) as conn:
    with conn.cursor() as cursor:
        for i in range(4000):

            profile = fake.polish_profile()
            phone = profile["phone_number"]
            first_name = profile["first_name"]
            last_name = profile["last_name"]
            email = profile["mail"]
            adress = profile["address"].replace("\n", ", ")
            gender = profile["sex"]

            family_member_profile = fake.polish_profile()
            family_member_phone = fake.polish_phone_number()
            family_member_email = family_member_profile["mail"]
            family_gender = family_member_profile["sex"]
            if family_gender == "M":
                family_relation = np.random.choice(
                    [
                        "brat",
                        "ojciec",
                        "syn",
                        "wujek",
                        "dziadek",
                        "kuzyn",
                        "wnuk",
                        "pradziadek",
                        "prawnuk",
                    ],
                    p=[0.2, 0.2, 0.2, 0.1, 0.1, 0.1, 0.05, 0.025, 0.025],
                )
            elif family_gender == "F":
                family_relation = np.random.choice(
                    [
                        "siostra",
                        "matka",
                        "córka",
                        "ciocia",
                        "babcia",
                        "kuzynka",
                        "wnuczka",
                        "prababcia",
                        "prawnuczka",
                    ],
                    p=[0.2, 0.2, 0.2, 0.1, 0.1, 0.1, 0.05, 0.025, 0.025],
                )
            else:
                family_relation = None

            cursor.execute(
                f"""INSERT INTO Uczestnicy (
                imie, 
                nazwisko, 
                telefon, 
                email, 
                plec,
                adres, 
                telefon_rodziny, 
                email_rodziny, 
                relacja_z_rodzina
                ) 
                VALUES (
                    '{first_name}', 
                    '{last_name}', 
                    '{phone}',
                    '{email}',
                    '{gender}',
                    '{adress}',
                    '{family_member_phone}',
                    '{family_member_email}',
                    '{family_relation}'
                    
                    );"""
            )
        conn.commit()

In [5]:
with mysql.connect(host=host, user=user, password=password, database=database) as conn:
    with conn.cursor() as cursor:
        themes_descriptions = {
            "Parkur po cudzych balkonach": "Uczestnicy muszą przebiec parkur po balkonach innych mieszkańców, nie zatrzymując się ani na chwilę. Punkty za przebiegłość i nieuchwytność.",
            "Wyścig na zegarach ściennych": "Uczestnicy muszą poruszać się na wielkich zegarach ściennych, które toczą się po podłodze. Punkty za czas i kreatywne użycie wskazówek.",
            "Zawody w tłumaczeniu języka pingwinów": "Każdy uczestnik otrzymuje nagranie z odgłosami pingwinów i musi wymyślić najbardziej przekonujące tłumaczenie. Bonus za dramatyczne historie.",
            "Sztafeta z kaktusem": "Uczestnicy muszą przekazywać sobie doniczkę z kaktusem, biegnąc przez tor przeszkód. Wygrywa drużyna, która zgubi najmniej kolców.",
            "Zawody w liczeniu kropli deszczu": "Uczestnicy muszą liczyć krople deszczu spadające na ich głowę. Wygrywa osoba, która zachowa największą powagę podczas zadania.",
            "Budowanie igloo z mydła w kostce": "Gra zespołowa, w której uczestnicy budują igloo wyłącznie z kostek mydła. Konstrukcje zazwyczaj rozjeżdżają się przed ukończeniem.",
            "Festiwal naśladowania dźwięków tostera": "Każdy uczestnik stara się jak najdokładniej odtworzyć dźwięk pracy tostera od startu aż po wyskakujące grzanki.",
            "Skakanie w workach na głowie": "Zawodnicy zakładają worki na głowę i próbują skakać, kierując się wyłącznie wskazówkami swoich partnerów. Widzenie niewskazane.",
            "Spacer z lodówką na smyczy": "Uczestnicy ciągną lodówki na smyczach, udając, że to ich pupile. Punkty za rozmowę z lodówką i jej `reakcje`.",
            "Składanie origami z tostów": "Konkurs na najpiękniejsze origami wykonane z ciepłych tostów. Wersje z serem uznawane są za poziom zaawansowany.",
        }
        for i, (theme, desc) in enumerate(themes_descriptions.items(), 1):

            mu = 200
            sigma = 20
            cost = np.random.normal(mu, sigma)
            cost = round(cost, 2)
            cost = np.clip(cost, 130, 270)

            cursor.execute(
                f"""INSERT INTO Tematyki (id_tematyki, nazwa, opis, koszt) 
                VALUES (
                    {i},
                    '{theme}', 
                    '{desc}',
                    '{cost}'
                    );"""
            )
    conn.commit()

In [6]:
forign_cities = {
    "Paryż": "Francja",
    "Lyon": "Francja",
    "Nicea": "Francja",
    "Marsylia": "Francja",
    "Bordeaux": "Francja",
    "Madryt": "Hiszpania",
    "Barcelona": "Hiszpania",
    "Sevilla": "Hiszpania",
    "Valencia": "Hiszpania",
    "Malaga": "Hiszpania",
    "Rzym": "Włochy",
    "Wenecja": "Włochy",
    "Florencja": "Włochy",
    "Mediolan": "Włochy",
    "Neapol": "Włochy",
    "Dubrownik": "Chorwacja",
    "Split": "Chorwacja",
    "Zagrzeb": "Chorwacja",
    "Rijeka": "Chorwacja",
    "Pula": "Chorwacja",
    "Kopenhaga": "Dania",
    "Aarhus": "Dania",
    "Odense": "Dania",
    "Aalborg": "Dania",
    "Esbjerg": "Dania",
}
with mysql.connect(host=host, user=user, password=password, database=database) as conn:
    with conn.cursor() as cursor:
        for city, countries in forign_cities.items():
            mu = 250
            sigma = 30
            cost = np.random.normal(mu, sigma)
            cost = round(cost, 2)
            cost = np.clip(cost, 200, 300)

            cursor.execute(
                f"""INSERT INTO Kierunki (miasto, koszt, kraj)
                VALUES (
                    '{city}',
                    '{cost}',
                    '{countries}'
                    );"""
            )
        conn.commit()

        for city in ["Warszawa", "Kraków", "Gdańsk", "Zakopane", "Poznań"]:
            mu = 50
            sigma = 10
            cost = np.random.normal(mu, sigma)
            cost = round(cost, 2)
            cost = np.clip(cost, 30, 70)

            cursor.execute(
                f"""INSERT INTO Kierunki (miasto, koszt, kraj)
                VALUES (
                    '{city}',
                    '{cost}',
                    'Polska'
                    );"""
            )
    conn.commit()

In [7]:
positions = [
    "Przewodnik",
    "Przewodnik",
    "Przewodnik",
    "Agent Biura Podróży",
    "Agent Biura Podróży",
    "Agent Biura Podróży",
    "Agent Biura Podróży",
    "Agent Biura Podróży",
    "Kierownik",
    "Kierownik",
]

with mysql.connect(host=host, user=user, password=password, database=database) as conn:
    with conn.cursor() as cursor:
        for i in range(len(positions)):

            polish_characters = {
                "ą": "a",
                "ć": "c",
                "ę": "e",
                "ł": "l",
                "ń": "n",
                "ó": "o",
                "ś": "s",
                "ź": "z",
                "ż": "z",
                "Ą": "A",
                "Ć": "C",
                "Ę": "E",
                "Ł": "L",
                "Ń": "N",
                "Ó": "O",
                "Ś": "S",
                "Ź": "Z",
                "Ż": "Z",
            }
            profile = fake.polish_profile()
            phone = fake.polish_phone_number()
            first_name = profile["first_name"]
            last_name = profile["last_name"]
            first_name = "".join(polish_characters.get(c, c) for c in first_name)
            last_name = "".join(polish_characters.get(c, c) for c in last_name)
            email = first_name.lower() + "." + last_name.lower() + "@biuro.frajdy.pl"
            date_of_employment = fake.date_between(start_date="-4y", end_date="-1m")

            position = positions.pop()

            match position:
                case "Przewodnik":
                    salery = np.random.normal(7000, 200)
                    salery = round(salery, -1)
                    salery = np.clip(salery, 6000, 9000)
                case "Agent Biura Podróży":
                    salery = np.random.normal(6000, 200)
                    salery = round(salery, -1)
                    salery = np.clip(salery, 5000, 7000)
                case "Kierownik":
                    salery = np.random.normal(8500, 200)
                    salery = round(salery, -1)
                    salery = np.clip(salery, 7000, 10000)

            cursor.execute(
                f"""INSERT INTO Pracownicy (imie, nazwisko, telefon, email, data_zatrudnienia, wynagrodzenie, stanowisko) 
                VALUES (
                    '{first_name}', 
                    '{last_name}', 
                    '{phone}',
                    '{email}',
                    '{date_of_employment}',
                    '{salery}',
                    '{position}'
                    );"""
            )

        cursor.execute(
            f"""
        INSERT INTO Pracownicy (imie, nazwisko, telefon, email, data_zatrudnienia, wynagrodzenie, stanowisko)
        VALUES ('Dominik', 'Jur', '{fake.polish_phone_number()}', 'jur.dominik@biuro.frajdy.pl', '2015-01-03', 10020, 'Prezes'),
        ('Szymon', 'Klim', '{fake.polish_phone_number()}', 'szymon.klim@biuro.frajdy.pl', '2021-01-14', 7870, 'Księgowy'),
        ('Kacper', 'Samulski', '{fake.polish_phone_number()}', 'kacper.samulski@biuro.frajdy.pl', '2016-03-03', 9070, 'Analityk danych'),
        ('Albert', 'Janik', '{fake.polish_phone_number()}', 'albert.janik@biuro.frajdy.pl', '2016-02-04', 9070, 'Wiceprezes')
                """
        )
    conn.commit()

In [8]:
from calendar import monthrange
from datetime import datetime, timedelta, date


def generate_biased_date(year_range, count=1):
    dates = []
    for _ in range(count):

        mu = 7
        sigma = 3
        month = np.random.normal(mu, sigma)
        month = round(month)
        month = np.clip(month, 1, 12)

        start_year, end_year = year_range
        years = list(range(start_year, end_year + 1))
        weights = [1.2 ** (i - start_year) for i in years]
        total_weight = sum(weights)
        normalized_weights = [w / total_weight for w in weights]
        year = np.random.choice(years, p=normalized_weights)

        _, last_day = monthrange(year, month)
        day = random.randint(1, last_day)
        date_ = datetime(year, month, day)

        if count == 1:
            return date_

        dates.append(date_)

    return dates

In [9]:
with mysql.connect(host=host, user=user, password=password, database=database) as conn:
    with conn.cursor() as cursor:

        cursor.execute(
            "SELECT id_pracownika FROM Pracownicy WHERE stanowisko = 'Przewodnik';"
        )
        tour_guide_ids = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT id_kierunku, kraj FROM Kierunki;")
        poland_dest_ids = []
        france_dest_ids = []
        croatia_dest_ids = []
        italy_dest_ids = []
        spain_dest_ids = []
        denmark_dest_ids = []

        dests = cursor.fetchall()
        for row in dests:
            if row[1] == "Polska":
                poland_dest_ids.append(row[0])
            elif row[1] == "Francja":
                france_dest_ids.append(row[0])
            elif row[1] == "Chorwacja":
                croatia_dest_ids.append(row[0])
            elif row[1] == "Włochy":
                italy_dest_ids.append(row[0])
            elif row[1] == "Hiszpania":
                spain_dest_ids.append(row[0])
            elif row[1] == "Dania":
                denmark_dest_ids.append(row[0])

        countries_ids = {
            "Polska": poland_dest_ids,
            "Francja": france_dest_ids,
            "Chorwacja": croatia_dest_ids,
            "Włochy": italy_dest_ids,
            "Hiszpania": spain_dest_ids,
            "Dania": denmark_dest_ids,
        }

        already_seen = set()
        i = 0
        while i < 4000:
            i += 1

            country = np.random.choice(
                ["Polska", "Francja", "Chorwacja", "Włochy", "Hiszpania", "Dania"],
                p=[0.45, 0.05, 0.2, 0.15, 0.1, 0.05],
            )
            dest = np.random.choice(countries_ids[country])
            theme = np.random.choice(
                range(1, 11),
                p=[0.2, 0.125, 0.125, 0.1, 0.1, 0.1, 0.075, 0.075, 0.05, 0.05],
            )
            start_date = generate_biased_date((2015, 2024), count=1)
            end_date = start_date + timedelta(
                days=int(np.clip(round(np.random.normal(7, 3)), 1, 18))
            )
            total_days = (end_date - start_date).days
            number_of_participants = np.clip(np.random.poisson(4), 1, 30)
            tour_guide_id = np.random.choice(tour_guide_ids)
            cursor.execute(
                f"""(SELECT (k.koszt+(t.koszt * {total_days}))*{number_of_participants} as koszt
                    FROM Kierunki k
                        JOIN Tematyki t 
                        ON k.id_kierunku = '{dest}' AND t.id_tematyki = '{theme}'
                    LIMIT 1);"""
            )
            cost = float([row[0] for row in cursor.fetchall()][0])
            cursor.execute(
                f"""INSERT INTO Wycieczki (
                    id_wycieczki,
                    id_tematyki,
                    id_kierunku,
                    id_pracownika,
                    data_rozpoczecia,
                    data_zakonczenia,
                    koszt_organizacji,
                    ilosc_uczestnikow
                    )
                
                VALUES (
                    '{i}',
                    '{theme}',
                    '{dest}',
                    {tour_guide_id}, 
                    '{start_date}',
                    '{end_date}',
                    '{cost}',
                    '{number_of_participants}'
                    
                    );"""
            )
            conn.commit()

            cursor.execute("SELECT id_uczestnika FROM Uczestnicy;")
            participant_ids = [row[0] for row in cursor.fetchall()]

            already_on_trip = set()
            while len(already_on_trip) < number_of_participants:
                participant_id = np.random.choice(participant_ids)
                if participant_id not in already_on_trip:

                    rated = np.random.rand() > 0.6
                    rating = (
                        np.random.choice(
                            [1, 2, 3, 4, 5], p=[0.1, 0.03, 0.03, 0.3, 0.54]
                        )
                        if rated
                        else None
                    )

                    cursor.execute(
                        f"""INSERT INTO Wycieczki_Uczestnicy (
                        id_uczestnika,
                        id_wycieczki,
                        kupujacy,
                        ocena
                        )
                    VALUES (
                        '{participant_id}',
                        '{i}',
                        '{int(len(already_on_trip) == 0)}',
                        {rating if rating else 'NULL'}
                        );"""
                    )

                    already_on_trip.add(participant_id)
                    already_seen.add(participant_id)

            conn.commit()

            cursor.execute(
                "SELECT id_pracownika FROM Pracownicy WHERE stanowisko = 'Agent Biura Podróży';"
            )
            agent_ids = [row[0] for row in cursor.fetchall()]
            agent = np.random.choice(agent_ids)
            price = cost * np.clip(np.random.normal(1.6, 0.1), 1.1, 2.0)
            transaction_type = np.random.choice(
                ["karta platnicza", "gotówka", "przelew", "paypal", "blik"],
                p=[0.7, 0.1, 0.05, 0.05, 0.1],
            )
            transaction_date = fake.date_between_dates(
                date_start=start_date - timedelta(days=700),
                date_end=start_date - timedelta(days=14),
            )

            if transaction_date < date(2015, 1, 3):
                transaction_date = date(2015, 1, 3)

            cursor.execute(
                f"""INSERT INTO Transakcje (
                id_uczestnika,
                id_pracownika,
                id_wycieczki,
                kwota,
                sposob_platnosci,
                data_transakcji
                ) 
                VALUES 
                (
                (SELECT id_uczestnika FROM Wycieczki_Uczestnicy WHERE id_wycieczki = '{i}' AND kupujacy = 1 LIMIT 1),
                '{agent}',
                '{i}',
                '{price}',
                '{transaction_type}',
                '{transaction_date}'
                );"""
            )

            conn.commit()

In [14]:
with mysql.connect(host=host, user=user, password=password, database=database) as conn:
    with conn.cursor() as cursor:
        with open("../sql/clear.sql", "r") as file:
            cursor.execute(file.read())
        conn.commit()